### RFM 분석 실습
- Recency: 얼마나 최근에 구매했는가
- Frequency: 얼마나 자주 구매했 (불가시 구매수량)는가
- Monetary: 얼마나 많은 금액을 지출했는가

### 고객 분석

> voice_no: 송장 번호. 명사 같은. 각 작업에 고유하게 할당된 문자 'I'와 6자리 정수의 조합  
customer_id: 고객번호. 명사 같은. 각 작업에 고유하게 할당된 문자 'C'와 6자리 정수의 조합  
gender: 고객 성별의 문자a열 변수  
age: 고객 연령의 양의 정수 변수  
category: 구매한 상품 카테고리의 문자열 변수 
quantity: 거래당 각 제품(항목)의 수량  
price: 단가. 숫자. 단위당 제품 가격은 터키 리라(TL)  
payment_method: 거래에 사용된 결제 방법(현금, 신용 카드 또는 직불 카드)의 문자열 변수  
invoice_date: 송장 날짜. 거래가 발생한 날  
shopping_mall: 거래가 이루어진 쇼핑몰 이름의 문자열 변수  

In [1]:
import pandas as pd
import numpy as np

co_df = pd.read_csv('./datasets/customer_shopping_data.csv')
co_df


,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


## 해야할일  
> 송장번호는 날립시다.  
> 날짜 변환하기 (Recency)  
> totalprice 만들기  

In [2]:
shopping_mall_values = co_df['category'].unique()
print(shopping_mall_values)
#  카테고리 8개 / 쇼핑몽 10개

['Clothing' 'Shoes' 'Books' 'Cosmetics' 'Food & Beverage' 'Toys'
 'Technology' 'Souvenir']


In [3]:
co_df.isna().sum()

invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
shopping_mall     0
dtype: int64

In [4]:
co_df.duplicated().sum()

0

In [5]:
duplicate_count = co_df.duplicated(subset=['customer_id']).sum()
print("중복된 고객 ID의 수:", duplicate_count)

중복된 고객 ID의 수: 0


In [6]:
age_counts_sorted = co_df['age'].value_counts().sort_index().head(10)
print(age_counts_sorted)


age
18    1844
19    1936
20    1844
21    1947
22    2051
23    1897
24    1977
25    1863
26    1896
27    1950
Name: count, dtype: int64


In [7]:
# Quantity 열에서 0 또는 음수가 있는지 확인
negative_quantity = co_df[co_df['quantity'] <= 0]
if negative_quantity.empty:
    print("Quantity 열에 음수나 0이 없습니다.")
else:
    print("음수나 0이 포함된 행:")
    print(negative_quantity)

# Price 열에서 0 또는 음수가 있는지 확인
negative_price = co_df[co_df['price'] <= 0]
if negative_price.empty:
    print("Price 열에 음수나 0이 없습니다.")
else:
    print("음수나 0이 포함된 행:")
    print(negative_price)


Quantity 열에 음수나 0이 없습니다.
Price 열에 음수나 0이 없습니다.


In [8]:
co_df = co_df.drop(columns=['invoice_no'])
co_df

,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...
99452,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


In [ ]:
import pandas as pd

# 'invoice_date' 열을 datetime 형식으로 변환
co_df['invoice_date'] = pd.to_datetime(co_df['invoice_date'], format="%d/%m/%Y")

# 결과 출력

co_df.loc[:, 'Recency'] = pd.to_datetime(co_df['invoice_date'])\
                    .apply(lambda x: (pd.to_datetime(co_df['invoice_date'].max()) - x).days)
co_df

In [ ]:
sorted_co_df = co_df.sort_values(by='Recency', ascending=False)
sorted_co_df

In [ ]:
co_df = pd.DataFrame(co_df)
co_df['totalPrice'] = co_df['quantity'] * co_df['price']
co_df

In [ ]:
rfm = co_df[['Recency', 'quantity', 'totalPrice']].copy()

# 새로운 데이터프레임의 열 이름 변경
rfm.columns = ['Recency', 'Frequency', 'Monetary']

rfm

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
rfm[['Recency', 'Frequency', 'Monetary']] = \
                                    scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])
# 최근일 수록 숫자가 더 작게 나오기 때문에, 1을 빼준 값으로 점수를 계산한다.
rfm['Recency'] = 1 - rfm['Recency']
rfm

In [ ]:
rfm[['Recency', 'Frequency', 'Monetary']].mean()



In [ ]:
co_df['TotalScore'] = rfm['Recency'] + rfm['Frequency'] + rfm['Monetary']
co_df

In [ ]:
import numpy as np

l1, l2, l3, l4 = np.percentile(co_df['TotalScore'], [25, 45, 65, 85])
print(l1, l2, l3, l4)

In [ ]:
def get_level(x):
    if x <= l1:
        return 5
    if x > l1 and x <= l2:
        return 4
    if x > l2 and x <= l3:
        return 3
    if x > l3 and x <= l4:
        return 2
    return 1

In [ ]:
co_df['Level'] = co_df['TotalScore'].apply(get_level)
co_df['Level'] = co_df['Level'].replace([5, 4, 3, 2, 1], ['Bronze', 'Silver', 'Gold', 'Diamond', 'VIP'])
print(co_df['Level'].value_counts())
co_df

In [ ]:
total_price_by_mall = co_df.groupby('shopping_mall')['totalPrice'].sum()

# 결과 출력
print(total_price_by_mall)

import matplotlib.pyplot as plt

# 'shopping_mall' 별로 거래 횟수 계산
transaction_count_by_mall = co_df.groupby('shopping_mall').size()

# 'shopping_mall' 별로 계산된 총 판매액과 거래 횟수를 합침
combined_data = pd.concat([total_price_by_mall, transaction_count_by_mall], axis=1)
combined_data.columns = ['Total Price', 'Transaction Count']

# 'Total Price' 열을 기준으로 내림차순 정렬
combined_data_sorted = combined_data.sort_values(by='Total Price', ascending=False)

# 막대 그래프로 시각화
combined_data_sorted.plot(kind='bar', secondary_y='Transaction Count', rot=45)
plt.title('Total Sales and Transaction Count by Shopping Mall')
plt.xlabel('Shopping Mall')
plt.ylabel('Total Price')
plt.show()


In [ ]:
# 각 고객의 나이대를 할당하는 함수
def assign_age_group(age):
    if 18 <= age <= 29:
        return '18-29'
    elif 30 <= age <= 39:
        return '30-39'
    elif 40 <= age <= 49:
        return '40-49'
    elif 50 <= age <= 59:
        return '50-59'
    elif 60 <= age <= 70:
        return '60-70'
    # 이하 생략 (나머지 나이대에 대해서도 동일한 방식으로 처리)

# 'AgeGroup' 열 추가
co_df['AgeGroup'] = co_df['age'].apply(assign_age_group)

# 각 나이대별로 'totalPrice'의 합 구하기
total_price_by_age_group = co_df.groupby('AgeGroup')['totalPrice'].sum()

# 결과 출력
print(total_price_by_age_group)

In [ ]:
import matplotlib.pyplot as plt

# 각 나이대별 고객 수 계산
customer_count_by_age_group = co_df['AgeGroup'].value_counts()

# 나이대를 순서대로 정렬하여 데이터 정렬
total_price_by_age_group = total_price_by_age_group.sort_index()
customer_count_by_age_group = customer_count_by_age_group.sort_index()

# 그래프 크기 지정
plt.figure(figsize=(10, 5))

# 첫 번째 막대 그래프: Total Price
plt.subplot(2, 1, 1)
plt.bar(total_price_by_age_group.index, total_price_by_age_group, color='skyblue')
plt.title('Total Sales by Age Group')
plt.ylabel('Total Price')

# 두 번째 막대 그래프: Customer Count
plt.subplot(2, 1, 2)
plt.bar(customer_count_by_age_group.index, customer_count_by_age_group, color='orange')
plt.xlabel('Age Group')
plt.ylabel('Customer Count')

# 그래프 간 간격 조정
plt.tight_layout()

# 그래프 표시
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# AgeGroup과 shopping_mall을 기준으로 그룹화하여 각 그룹의 크기를 세기
transaction_count_by_age_group_and_mall = co_df.groupby(['AgeGroup', 'shopping_mall']).size()

# 피벗 테이블로 변환하여 각 나이대별 쇼핑몰 거래 횟수를 나타내는 형태로 가공
transaction_count_pivot = transaction_count_by_age_group_and_mall.unstack()

# 막대 그래프로 시각화
transaction_count_pivot.plot(kind='bar', stacked=True)
plt.title('Transaction Count by Age Group and Shopping Mall')
plt.xlabel('Age Group')
plt.ylabel('Transaction Count')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# AgeGroup과 shopping_mall을 기준으로 그룹화하여 각 그룹의 크기를 세기
transaction_count_by_age_group_and_mall = co_df.groupby(['AgeGroup', 'shopping_mall']).size()

# 피벗 테이블로 변환하여 각 나이대별 쇼핑몰 거래 횟수를 나타내는 형태로 가공
transaction_count_pivot = transaction_count_by_age_group_and_mall.unstack()

# 각 쇼핑몰의 거래 횟수를 합산하여 큰 순서대로 정렬
transaction_count_pivot_sorted = transaction_count_pivot.sum(axis=0).sort_values(ascending=False)

# 정렬된 순서에 따라 데이터를 선택하여 재배열
transaction_count_pivot_sorted = transaction_count_pivot[transaction_count_pivot_sorted.index]

# 막대 그래프로 시각화 (x와 y 축을 바꾸기 위해 .T를 사용하여 전치(transpose) 수행)
transaction_count_pivot_sorted.T.plot(kind='bar', stacked=True)
plt.title('Transaction Count by Shopping Mall and Age Group')  # 타이틀 변경
plt.xlabel('Shopping Mall')  # x축 라벨 변경
plt.ylabel('Transaction Count')  # y축 라벨 변경
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Clothing', 'Shoes', 'Technology' 카테고리의 totalPrice 합계
total_price_clothing_shoes_tech = co_df.loc[co_df['category'].isin(['Clothing', 'Shoes', 'Technology']), 'totalPrice'].sum()

# 나머지 카테고리의 totalPrice 합계
total_price_other = co_df.loc[~co_df['category'].isin(['Clothing', 'Shoes', 'Technology']), 'totalPrice'].sum()

# 원그래프로 시각화
plt.figure(figsize=(8, 8))
plt.pie([total_price_clothing_shoes_tech, total_price_other], labels=['Clothing, Shoes, Technology', 'Other'], autopct='%1.1f%%', startangle=140)
plt.title('Total Price by Category')
plt.axis('equal')  # 원 그래프가 원형으로 보이도록 설정
plt.show()


In [ ]:
# category 별 totalPrice 계산
total_price_by_category = co_df.groupby('category')['totalPrice'].sum()

# 결과 출력
print(total_price_by_category)


In [ ]:
import matplotlib.pyplot as plt

# 막대 그래프로 카테고리별 totalPrice 시각화
total_price_by_category.plot(kind='bar')
plt.title('Total Sales by Category')
plt.xlabel('Category')
plt.ylabel('Total Price')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 카테고리별 거래 횟수 계산
transaction_count_by_category = co_df.groupby('category').size()

# 막대 그래프로 시각화
transaction_count_by_category.plot(kind='bar', color='skyblue')
plt.title('Transaction Count by Category')
plt.xlabel('Category')
plt.ylabel('Transaction Count')
plt.xticks(rotation=45)  # x 축 라벨 회전
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 쇼핑몰별 Level 분포 계산
level_distribution_by_shopping_mall = co_df.groupby('shopping_mall')['Level'].value_counts()

# 쇼핑몰을 Level 분포 기준으로 큰 순서대로 정렬
sorted_shopping_malls = level_distribution_by_shopping_mall.groupby(level=0).sum().sort_values(ascending=False).index

# 정렬된 쇼핑몰 순서에 따라 데이터 선택하여 재배열
level_distribution_sorted = level_distribution_by_shopping_mall.unstack().loc[sorted_shopping_malls]

# 막대 그래프로 시각화
level_distribution_sorted.plot(kind='bar', stacked=True)
plt.title('Level Distribution by Shopping Mall')
plt.xlabel('Shopping Mall')
plt.ylabel('Count')
plt.xticks(rotation=45)  # x 축 라벨 회전
plt.legend(title='Level')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 쇼핑몰별 Level 분포 계산
level_distribution_by_shopping_mall = co_df.groupby('category')['Level'].value_counts()

# 쇼핑몰을 Level 분포 기준으로 큰 순서대로 정렬
sorted_shopping_malls = level_distribution_by_shopping_mall.groupby(level=0).sum().sort_values(ascending=False).index

# 정렬된 쇼핑몰 순서에 따라 데이터 선택하여 재배열
level_distribution_sorted = level_distribution_by_shopping_mall.unstack().loc[sorted_shopping_malls]

# 막대 그래프로 시각화
level_distribution_sorted.plot(kind='bar', stacked=True)
plt.title('Level Distribution by Shopping Mall')
plt.xlabel('Shopping Mall')
plt.ylabel('Count')
plt.xticks(rotation=45)  # x 축 라벨 회전
plt.legend(title='Level')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Level별 Total Price 합계 계산
total_price_by_level = co_df.groupby('Level')['totalPrice'].sum()

# Level별 Total Price 합계를 내림차순으로 정렬
total_price_by_level_sorted = total_price_by_level.sort_values(ascending=False)

# 막대 그래프로 시각화
total_price_by_level_sorted.plot(kind='bar', color='skyblue')
plt.title('Total Price by Level')
plt.xlabel('Level')
plt.ylabel('Total Price')
plt.xticks(rotation=0)  # x 축 라벨 회전
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# VIP 등급에 해당하는 행들만 필터링
vip_data = co_df[co_df['Level'] == 'VIP']

# VIP 등급에 해당하는 데이터를 기준으로 AgeGroup을 그룹화하여 각 그룹의 크기를 세기
grouped = vip_data.groupby('AgeGroup').size()

# 막대 그래프로 시각화
grouped.plot(kind='bar', color='skyblue')
plt.title('VIP Distribution by AgeGroup')
plt.xlabel('AgeGroup')
plt.ylabel('Count')
plt.xticks(rotation=0)  # x 축 라벨 회전
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Clothing, Shoes, Technology 카테고리에 해당하는 행들만 필터링
filtered_data = co_df[co_df['category'].isin(['Clothing', 'Shoes', 'Technology'])]

# 쇼핑몰과 카테고리를 기준으로 그룹화하여 총 가격을 계산
grouped_data = filtered_data.groupby(['shopping_mall', 'category'])['totalPrice'].sum().unstack()

# 쇼핑몰별 총 가격의 합을 계산하여 큰 순서대로 정렬
sorted_shopping_malls = grouped_data.sum(axis=1).sort_values(ascending=False).index

# 큰 순서대로 정렬된 쇼핑몰에 맞게 데이터를 정렬
grouped_data_sorted = grouped_data.loc[sorted_shopping_malls]

# 그룹화된 데이터를 바탕으로 그래프 생성
grouped_data_sorted.plot(kind='bar', figsize=(10, 6))
plt.title('Total Price by Shopping Mall and Category')
plt.xlabel('Shopping Mall')
plt.ylabel('Total Price')
plt.xticks(rotation=45)
plt.legend(title='Category')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Clothing, Shoes, Technology 카테고리에 해당하는 행들만 필터링
filtered_data = co_df[co_df['category'].isin(['Clothing', 'Shoes', 'Technology'])]

# AgeGroup과 카테고리를 기준으로 그룹화하여 각 그룹의 크기를 세기
grouped_count = filtered_data.groupby(['AgeGroup', 'category']).size()

# 그룹화된 데이터를 피벗 테이블로 변환하여 카테고리별로 열을 나타내는 형태로 변환
grouped_pivot = grouped_count.unstack()

# 막대 그래프로 시각화
grouped_pivot.plot(kind='bar', figsize=(10, 6))
plt.title('Category Count by AgeGroup')
plt.xlabel('AgeGroup')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.legend(title='Category')
plt.grid(True)
plt.show()


In [ ]:
# 각각의 카테고리에 대해 VIP 등급의 totalprice 평균 구하기
average_total_price_vip_clothing = co_df[(co_df['Level'] == 'VIP') & (co_df['category'] == 'Clothing')]['price'].mean()
average_total_price_vip_shoes = co_df[(co_df['Level'] == 'VIP') & (co_df['category'] == 'Shoes')]['price'].mean()
average_total_price_vip_technology = co_df[(co_df['Level'] == 'VIP') & (co_df['category'] == 'Technology')]['price'].mean()

print("VIP 등급 중 Clothing 카테고리의 totalprice 평균:", average_total_price_vip_clothing)
print("VIP 등급 중 Shoes 카테고리의 totalprice 평균:", average_total_price_vip_shoes)
print("VIP 등급 중 Technology 카테고리의 totalprice 평균:", average_total_price_vip_technology)


In [ ]:
# 전체 데이터에서 category가 'Clothing', 'Shoes', 'Technology'인 행들을 필터링
total_clothing_data = co_df[co_df['category'] == 'Clothing']
total_shoes_data = co_df[co_df['category'] == 'Shoes']
total_tech_data = co_df[co_df['category'] == 'Technology']

# 필터링된 데이터에서 totalprice의 평균 계산
average_total_price_clothing = total_clothing_data['price'].mean()
average_total_price_shoes = total_shoes_data['price'].mean()
average_total_price_tech = total_tech_data['price'].mean()

print("전체 데이터 중 Clothing 카테고리의 totalprice 평균:", average_total_price_clothing)
print("전체 데이터 중 Shoes 카테고리의 totalprice 평균:", average_total_price_shoes)
print("전체 데이터 중 Technology 카테고리의 totalprice 평균:", average_total_price_tech)


In [ ]:
import matplotlib.pyplot as plt

# 카테고리 리스트
categories = ['Clothing', 'Shoes', 'Technology']

# VIP 등급 중 Clothing 카테고리의 totalprice 평균
average_total_price_clothing_vip = 1385.9008947504524
# VIP 등급 중 Shoes 카테고리의 totalprice 평균
average_total_price_shoes_vip = 2701.901354500437
# VIP 등급 중 Technology 카테고리의 totalprice 평균
average_total_price_tech_vip = 4539.388992537313

# 전체 데이터 중 Clothing 카테고리의 totalprice 평균
average_total_price_clothing_total = 901.0840212253893
# 전체 데이터 중 Shoes 카테고리의 totalprice 평균
average_total_price_shoes_total = 1807.3885678692443
# 전체 데이터 중 Technology 카테고리의 totalprice 평균
average_total_price_tech_total = 3156.935548438751

# 전체 데이터에서 각 카테고리의 totalprice 평균
total_price_means_total = [average_total_price_clothing_total, average_total_price_shoes_total, average_total_price_tech_total]

# VIP 등급 데이터에서 각 카테고리의 totalprice 평균
total_price_means_vip = [average_total_price_clothing_vip, average_total_price_shoes_vip, average_total_price_tech_vip]

# 막대 그래프로 시각화
plt.figure(figsize=(10, 6))
plt.bar(categories, total_price_means_total, color='skyblue', label='all data')
plt.bar(categories, total_price_means_vip, color='orange', label='VIP', alpha=0.5)  # VIP 등급을 다른 색상으로 표시
plt.title('Average Total Price by Category')
plt.xlabel('Category')
plt.ylabel('Average Total Price')
plt.legend()
plt.show()


In [ ]:
# 전체 데이터에서 category가 'Clothing', 'Shoes', 'Technology'인 행들을 필터링
total_clothing_data = co_df[co_df['category'] == 'Clothing']
total_shoes_data = co_df[co_df['category'] == 'Shoes']
total_tech_data = co_df[co_df['category'] == 'Technology']

# 필터링된 데이터에서 quantity의 평균 계산
average_quantity_clothing = total_clothing_data['quantity'].mean()
average_quantity_shoes = total_shoes_data['quantity'].mean()
average_quantity_tech = total_tech_data['quantity'].mean()

print("전체 데이터 중 Clothing 카테고리의 quantity 평균:", average_quantity_clothing)
print("전체 데이터 중 Shoes 카테고리의 quantity 평균:", average_quantity_shoes)
print("전체 데이터 중 Technology 카테고리의 quantity 평균:", average_quantity_tech)


In [ ]:
# VIP 등급 데이터에서 category가 'Clothing', 'Shoes', 'Technology'인 행들을 필터링
vip_clothing_data = co_df[(co_df['Level'] == 'VIP') & (co_df['category'] == 'Clothing')]
vip_shoes_data = co_df[(co_df['Level'] == 'VIP') & (co_df['category'] == 'Shoes')]
vip_tech_data = co_df[(co_df['Level'] == 'VIP') & (co_df['category'] == 'Technology')]

# 필터링된 데이터에서 quantity의 평균 계산
average_quantity_vip_clothing = vip_clothing_data['quantity'].mean()
average_quantity_vip_shoes = vip_shoes_data['quantity'].mean()
average_quantity_vip_tech = vip_tech_data['quantity'].mean()

print("VIP 등급 중 Clothing 카테고리의 quantity 평균:", average_quantity_vip_clothing)
print("VIP 등급 중 Shoes 카테고리의 quantity 평균:", average_quantity_vip_shoes)
print("VIP 등급 중 Technology 카테고리의 quantity 평균:", average_quantity_vip_tech)


In [ ]:
import matplotlib.pyplot as plt

# 카테고리 리스트
categories = ['Clothing', 'Shoes', 'Technology']

# 전체 데이터에서 각 카테고리의 quantity 평균
quantity_means_total = [3.0028126540435527, 3.0114610324895357, 3.006605284227382]

# VIP 등급 데이터에서 각 카테고리의 quantity 평균
quantity_means_vip = [4.618438065683993, 4.501893387707544, 4.323227611940299]

# 막대 그래프로 시각화
plt.figure(figsize=(10, 6))
plt.bar(categories, quantity_means_total, color='skyblue', label='all data')
plt.bar(categories, quantity_means_vip, color='orange', label='VIP', alpha=0.5)  # VIP 등급을 다른 색상으로 표시
plt.title('Average Quantity by Category')
plt.xlabel('Category')
plt.ylabel('Average Quantity')
plt.legend()
plt.show()


In [ ]:
category_counts = co_df.groupby(['AgeGroup', 'category']).size().unstack(fill_value=0)[['Clothing', 'Shoes', 'Technology']]

# 결과 출력
display(category_counts)

In [ ]:
import matplotlib.pyplot as plt

# 'Clothing', 'Shoes', 'Technology' 카테고리에 해당하는 거래만 필터링
filtered_df = co_df[co_df['category'].isin(['Clothing', 'Shoes', 'Technology'])]

# 쇼핑몰 별로 카테고리별 거래 횟수 계산
category_counts_by_mall = filtered_df.groupby(['shopping_mall', 'category']).size().unstack(fill_value=0)

# 각 쇼핑몰의 카테고리별 거래 횟수 합계 계산
total_counts_by_mall = category_counts_by_mall.sum(axis=1)

# 거래 횟수가 많은 순으로 정렬
total_counts_by_mall = total_counts_by_mall.sort_values(ascending=False)

# 정렬된 순서대로 쇼핑몰을 다시 정렬
sorted_category_counts_by_mall = category_counts_by_mall.loc[total_counts_by_mall.index]

# 그래프 생성
plt.figure(figsize=(12, 8))  # 그래프 크기 설정
sorted_category_counts_by_mall.plot(kind='bar', stacked=True)  # 막대 그래프 생성
plt.title('Category Counts by Shopping Mall')  # 그래프 제목 설정
plt.xlabel('Shopping Mall')  # x축 라벨 설정
plt.ylabel('Counts')  # y축 라벨 설정
plt.xticks(rotation=45)  # x축 라벨 회전
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')  # 범례 추가
plt.tight_layout()  # 그래프 여백 조정
plt.show()  # 그래프 출력


In [ ]:
import matplotlib.pyplot as plt

average_price_by_mall_category = filtered_df.groupby(['shopping_mall', 'category'])['totalPrice'].mean().unstack()
sorted_average_price_clothing = average_price_by_mall_category['Clothing'].sort_values(ascending=False)
sorted_average_price_shoes = average_price_by_mall_category['Shoes'].sort_values(ascending=False)
sorted_average_price_technology = average_price_by_mall_category['Technology'].sort_values(ascending=False)

# 그래프 생성
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 15))  # 3개의 서브플롯을 가진 그래프 생성

# Clothing 카테고리 그래프
sorted_average_price_clothing.plot(kind='bar', color='blue', ax=axes[0])  # 첫 번째 서브플롯에 그래프 생성
axes[0].set_title('Average TotalPrice of Clothing by Shopping Mall')  # 제목 설정
axes[0].set_xlabel('Shopping Mall')  # x축 라벨 설정
axes[0].set_ylabel('Average TotalPrice')  # y축 라벨 설정
axes[0].tick_params(axis='x', rotation=45)  # x축 라벨 회전

# Shoes 카테고리 그래프
sorted_average_price_shoes.plot(kind='bar', color='green', ax=axes[1])  # 두 번째 서브플롯에 그래프 생성
axes[1].set_title('Average TotalPrice of Shoes by Shopping Mall')  # 제목 설정
axes[1].set_xlabel('Shopping Mall')  # x축 라벨 설정
axes[1].set_ylabel('Average TotalPrice')  # y축 라벨 설정
axes[1].tick_params(axis='x', rotation=45)  # x축 라벨 회전

# Technology 카테고리 그래프
sorted_average_price_technology.plot(kind='bar', color='orange', ax=axes[2])  # 세 번째 서브플롯에 그래프 생성
axes[2].set_title('Average TotalPrice of Technology by Shopping Mall')  # 제목 설정
axes[2].set_xlabel('Shopping Mall')  # x축 라벨 설정
axes[2].set_ylabel('Average TotalPrice')  # y축 라벨 설정
axes[2].tick_params(axis='x', rotation=45)  # x축 라벨 회전

plt.tight_layout()  # 그래프 여백 조정
plt.show()  # 그래프 출력


In [ ]:
# 그래프 생성
agegroup_counts_by_mall = co_df.groupby('shopping_mall')['AgeGroup'].value_counts().unstack(fill_value=0)
agegroup_counts_by_mall = agegroup_counts_by_mall.sum(axis=1)  # 각 쇼핑몰별 총 인원
agegroup_counts_by_mall = agegroup_counts_by_mall.sort_values(ascending=False)

# plt.figure(figsize=(12, 8))  # 그래프 크기 설정
# agegroup_counts_by_mall.plot(kind='bar', stacked=True)  # 바 그래프 생성
# plt.title('Count of AgeGroup by Shopping Mall')  # 그래프 제목 설정
# plt.xlabel('Shopping Mall')  # x축 라벨 설정
# plt.ylabel('Count')  # y축 라벨 설정
# plt.xticks(rotation=45)  # x축 라벨 회전
# plt.legend(title='AgeGroup', bbox_to_anchor=(1.05, 1), loc='upper left')  # 범례 추가
# plt.tight_layout()  # 그래프 여백 조정
# plt.show()  # 그래프 출력
agegroup_counts_by_mall

In [ ]:
# AgeGroup 별로 VIP 비중 계산
vip_percentage_by_agegroup = co_df[co_df['Level'] == 'VIP'].groupby('AgeGroup')['Level'].count() / co_df.groupby('AgeGroup')['Level'].count()

# 그래프 생성
plt.figure(figsize=(10, 6))  # 그래프 크기 설정
vip_percentage_by_agegroup.plot(kind='bar', color='skyblue')  # 바 그래프 생성
plt.title('VIP Percentage by AgeGroup')  # 그래프 제목 설정
plt.xlabel('AgeGroup')  # x축 라벨 설정
plt.ylabel('VIP Percentage')  # y축 라벨 설정
plt.xticks(rotation=45)  # x축 라벨 회전
plt.ylim(0, 1)  # y축 범위 설정
plt.tight_layout()  # 그래프 여백 조정
plt.show()  # 그래프 출력


In [ ]:
# Level이 VIP인 사람들의 AgeGroup 별 totalprice 계산
total_price_vip_by_agegroup = co_df[co_df['Level'] == 'VIP'].groupby('AgeGroup')['totalPrice'].sum()

# 그래프 생성
plt.figure(figsize=(10, 6))  # 그래프 크기 설정
total_price_vip_by_agegroup.plot(kind='bar', color='salmon')  # 바 그래프 생성
plt.title('Total Price by AgeGroup for VIP Customers')  # 그래프 제목 설정
plt.xlabel('AgeGroup')  # x축 라벨 설정
plt.ylabel('Total Price')  # y축 라벨 설정
plt.xticks(rotation=45)  # x축 라벨 회전
plt.tight_layout()  # 그래프 여백 조정
plt.show()  # 그래프 출력


In [ ]:
# AgeGroup이 '18-29'이고 Level이 'VIP'인 데이터 필터링
vip_in_agegroup_18_29 = co_df[(co_df['AgeGroup'] == '18-29') & (co_df['Level'] == 'VIP')]

# 전체 데이터 중에서 해당 비율 계산
vip_percentage_in_agegroup_18_29 = len(vip_in_agegroup_18_29) / len(co_df[co_df['AgeGroup'] == '18-29'])

# 결과 출력
print("AgeGroup이 '18-29'이고 Level이 'VIP'인 비율:", vip_percentage_in_agegroup_18_29)


In [ ]:
# 전체 고객 중에서 Level이 'VIP'인 데이터 필터링
vip_customers = co_df[co_df['Level'] == 'VIP']

# 전체 데이터 중에서 해당 비율 계산
vip_percentage_total = len(vip_customers) / len(co_df)

# 결과 출력
print("전체 고객 중 VIP 고객의 비율:", vip_percentage_total)


### 마케팅 전략 제안


> 전체 물품대금 중 95%는 옷,신발, 전자기기이다.  
> 가격에 대한 영향이 적고 젊은 층의 비율이 많을수록 많은 관심을 보이고있다. MD들의 유행을 따라가는 능력이 판매량을 좌우한다.  
> VIP의 연령대는 비슷하지만 지갑을 여는고객은 20대 고객이다.  
> 젊은 VIP를 위한 마케팅을 진행하고 고객이탈을 막아야한다.  
  
